<a href="https://colab.research.google.com/github/cengelhar/LearningNode/blob/master/Bicycle_Categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import json
from collections import OrderedDict

from PIL import Image

# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


###Initialize Model

In [0]:
data_dir = 'drive/My Drive/Bike Categorizing/Project/bikes'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'


train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)



train_loaders = torch.utils.data.DataLoader(train_dataset, batch_size=24)
valid_loaders = torch.utils.data.DataLoader(valid_dataset, batch_size=24)
test_loaders = torch.utils.data.DataLoader(test_dataset, batch_size=24)

#make sure that I assign my device according to whether cuda (GPU) is available. otherwise, use cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#define my model from torchVision
model = models.vgg19(pretrained=True)

#stop backpropagation across all parameters(especially features)
for param in model.parameters():
    param.requires_grad = False
    
#define classifier parameters in new model to fit out 102 output - 25088 inputs and 102 ouputs.
classifier = nn.Sequential(OrderedDict([
                         ('fc1',   nn.Linear(25088, 1024)),
                         ('drop',  nn.Dropout(p=0.2)),
                         ('relu',  nn.ReLU()),
                         ('fc2',   nn.Linear(1024, 512)),
                         ('drop',  nn.Dropout(p=0.2)),
                         ('relu',  nn.ReLU()),
                         ('fc3',   nn.Linear(512, 102)),
                         ('drop',  nn.Dropout(p=0.2)),
                         ('relu',  nn.ReLU()),
                         ('fc4',   nn.Linear(102, 3)),
                         ('drop',  nn.Dropout(p=0.2)),
                         ('relu',  nn.ReLU()),
                         ('output', nn.LogSoftmax(dim=1))
                         ]))

#update classifier parameters in model
model.classifier = classifier

#define cirteria to compare derived class to given label
criterion = nn.NLLLoss()

#define optimizer
optimizer = optim.SGD(model.classifier.parameters(), lr=0.05, momentum=0.0)

#Pass the appropriate processor based on the current device
model.to(device);

model_save_name = 'checkpoint-bike.pt'
path = F"/content/drive/My Drive/Google_Colab_Local/{model_save_name}"
checkpoint = torch.load(path)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.class_to_idx = checkpoint['class_to_idx']



###Training 1 - 24 epochs

In [0]:
min_loss = 20

#for i in content/drive/My Drive/Google_Colab_Local/workspace_utils.py.keep_awake(range(5)):

epochs = 24
steps = 0
accuracy = 0
running_loss = 0
#print_every = 50

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loaders:
        #increment steps by 1
        steps += 1

        #transfer inputs and labels to appropriate device (cuda or cpu)
        inputs, labels = inputs.to(device), labels.to(device)

        #zero out optimize gradients
        optimizer.zero_grad()

        #feedforward through network and get result
        logps = model(inputs)

        #calculate loss
        loss = criterion(logps, labels)

        #attribute the loss to individual weights
        loss.backward()

        #backpropagate to update the weights
        optimizer.step()

        #update the running_loss
        running_loss += loss.item()

        #display progess report while running
        total_steps = len(train_dataset)

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        print(f"Epoch {epoch+1}/{epochs}... ",
              f"Step {steps}/{total_steps}.. ",
              f"Step Loss {running_loss/steps:.3f}.. ",
              f"Accuracy {accuracy/steps:.3f}.. ")

    #Begin determining if a save is needed
    test_loss = 0
    iterations = 0
    model.eval()

    with torch.no_grad():
        for inputs, labels, in test_loaders:
            iterations += 1
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)

            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

        running_test_loss = test_loss/iterations

        if running_test_loss < min_loss:
            print((f"Minimum Loss This Epoch: {min_loss}...\n ",
                   f"Running Loss on Test Set: {running_test_loss}...\n ",
                   f"Minimum Loss has been updated... \n"
                   f"Model was Saved to 'checkpoint-bike.pt'"))

            min_loss = running_test_loss

            model_save_name = 'checkpoint-bike.pt'
            path = F"/content/drive/My Drive/Google_Colab_Local/{model_save_name}"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'class_to_idx': train_dataset.class_to_idx,
                }, path)

        else:
            print((f"Minimum Loss This Epoch: {min_loss}... ",
                   f"Running Loss on Test Set: {running_test_loss}... ",
                   f"Minimum Loss has NOT been updated... "
                   f"Model was NOT Saved"))             

    model.train()

Epoch 1/50...  Step 1/775..  Step Loss 1.159..  Accuracy 0.000.. 
Epoch 1/50...  Step 2/775..  Step Loss 0.987..  Accuracy 0.500.. 
Epoch 1/50...  Step 3/775..  Step Loss 0.841..  Accuracy 0.667.. 
Epoch 1/50...  Step 4/775..  Step Loss 0.752..  Accuracy 0.750.. 
Epoch 1/50...  Step 5/775..  Step Loss 0.659..  Accuracy 0.800.. 
Epoch 1/50...  Step 6/775..  Step Loss 0.584..  Accuracy 0.833.. 
Epoch 1/50...  Step 7/775..  Step Loss 0.525..  Accuracy 0.857.. 
Epoch 1/50...  Step 8/775..  Step Loss 0.473..  Accuracy 0.875.. 
Epoch 1/50...  Step 9/775..  Step Loss 0.429..  Accuracy 0.889.. 
Epoch 1/50...  Step 10/775..  Step Loss 0.400..  Accuracy 0.900.. 
Epoch 1/50...  Step 11/775..  Step Loss 0.371..  Accuracy 0.909.. 
Epoch 1/50...  Step 12/775..  Step Loss 0.345..  Accuracy 0.917.. 
Epoch 1/50...  Step 13/775..  Step Loss 0.320..  Accuracy 0.923.. 
Epoch 1/50...  Step 14/775..  Step Loss 0.301..  Accuracy 0.929.. 
Epoch 1/50...  Step 15/775..  Step Loss 0.284..  Accuracy 0.933.. 
Epoc

In [0]:
test_loss = 0
accuracy = 0

model.eval()
with torch.no_grad():
    for inputs, labels in valid_loaders:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()

        #calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(valid_loaders):.3f}.. ",
      f"Test accuracy: {accuracy/len(valid_loaders):.3f}")
model.train()

Test loss: 2.913..  Test accuracy: 0.323


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

### Training 2 -12 *epoch*s

In [0]:
min_loss = 20

#for i in content/drive/My Drive/Google_Colab_Local/workspace_utils.py.keep_awake(range(5)):

epochs = 25
steps = 0
accuracy = 0
running_loss = 0
#print_every = 50

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loaders:
        #increment steps by 1
        steps += 1

        #transfer inputs and labels to appropriate device (cuda or cpu)
        inputs, labels = inputs.to(device), labels.to(device)

        #zero out optimize gradients
        optimizer.zero_grad()

        #feedforward through network and get result
        logps = model(inputs)

        #calculate loss
        loss = criterion(logps, labels)

        #attribute the loss to individual weights
        loss.backward()

        #backpropagate to update the weights
        optimizer.step()

        #update the running_loss
        running_loss += loss.item()

        #display progess report while running
        total_steps = len(train_dataset)

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    print(f"Epoch {epoch+1}/{epochs}... ",
          f"Step {steps}/{total_steps}.. ",
          f"Step Loss {running_loss/steps:.3f}.. ",
          f"Accuracy {accuracy/steps:.3f}.. ")

    #Begin determining if a save is needed
    test_loss = 0
    iterations = 0
    model.eval()

    with torch.no_grad():
        for inputs, labels, in test_loaders:
            iterations += 1
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)

            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

        running_test_loss = test_loss/iterations

        if running_test_loss < min_loss:
            print((f"Minimum Loss This Epoch: {min_loss}...\n ",
                   f"Running Loss on Test Set: {running_test_loss}...\n ",
                   f"Minimum Loss has been updated... \n"
                   f"Model was Saved to 'checkpoint-bike.pt'"))

            min_loss = running_test_loss

            model_save_name = 'checkpoint-bike.pt'
            path = F"/content/drive/My Drive/Google_Colab_Local/{model_save_name}"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'class_to_idx': train_dataset.class_to_idx,
                }, path)

        else:
            print((f"Minimum Loss This Epoch: {min_loss}... ",
                   f"Running Loss on Test Set: {running_test_loss}... ",
                   f"Minimum Loss has NOT been updated... "
                   f"Model was NOT Saved"))             

    model.train()

Epoch 1/25...  Step 63/752..  Step Loss 0.286..  Accuracy 0.899.. 
('Minimum Loss This Epoch: 20...\n ', 'Running Loss on Test Set: 2.9706484377151354...\n ', "Minimum Loss has been updated... \nModel was Saved to 'checkpoint-bike.pt'")
Epoch 2/25...  Step 126/752..  Step Loss 0.283..  Accuracy 0.897.. 
('Minimum Loss This Epoch: 2.9706484377151354... ', 'Running Loss on Test Set: 3.0980420808424243... ', 'Minimum Loss has NOT been updated... Model was NOT Saved')
Epoch 3/25...  Step 189/752..  Step Loss 0.279..  Accuracy 0.903.. 
('Minimum Loss This Epoch: 2.9706484377151354... ', 'Running Loss on Test Set: 3.0935109168174675... ', 'Minimum Loss has NOT been updated... Model was NOT Saved')
Epoch 4/25...  Step 252/752..  Step Loss 0.279..  Accuracy 0.903.. 
('Minimum Loss This Epoch: 2.9706484377151354...\n ', 'Running Loss on Test Set: 2.536218400776852...\n ', "Minimum Loss has been updated... \nModel was Saved to 'checkpoint-bike.pt'")
Epoch 5/25...  Step 315/752..  Step Loss 0.275

KeyboardInterrupt: ignored

In [0]:
test_loss = 0
accuracy = 0

model.eval()
with torch.no_grad():
    for inputs, labels in valid_loaders:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()

        #calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(valid_loaders):.3f}.. ",
      f"Test accuracy: {accuracy/len(valid_loaders):.3f}")
model.train()

Test loss: 0.615..  Test accuracy: 0.735


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

###Training 3 - 21 epochs

In [0]:
min_loss = 20

#for i in content/drive/My Drive/Google_Colab_Local/workspace_utils.py.keep_awake(range(5)):

epochs = 25
steps = 0
accuracy = 0
running_loss = 0
#print_every = 50

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loaders:
        #increment steps by 1
        steps += 1

        #transfer inputs and labels to appropriate device (cuda or cpu)
        inputs, labels = inputs.to(device), labels.to(device)

        #zero out optimize gradients
        optimizer.zero_grad()

        #feedforward through network and get result
        logps = model(inputs)

        #calculate loss
        loss = criterion(logps, labels)

        #attribute the loss to individual weights
        loss.backward()

        #backpropagate to update the weights
        optimizer.step()

        #update the running_loss
        running_loss += loss.item()

        #display progess report while running
        total_steps = len(train_dataset)

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    print(f"Epoch {epoch+1}/{epochs}... ",
          f"Step {steps}/{total_steps}.. ",
          f"Step Loss {running_loss/steps:.3f}.. ",
          f"Accuracy {accuracy/steps:.3f}.. ")

    #Begin determining if a save is needed
    test_loss = 0
    iterations = 0
    model.eval()

    with torch.no_grad():
        for inputs, labels, in test_loaders:
            iterations += 1
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)

            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

        running_test_loss = test_loss/iterations

        if running_test_loss < min_loss:
            print((f"Minimum Loss This Epoch: {min_loss}...\n ",
                   f"Running Loss on Test Set: {running_test_loss}...\n ",
                   f"Minimum Loss has been updated... \n"
                   f"Model was Saved to 'checkpoint-bike.pt'"))

            min_loss = running_test_loss

            model_save_name = 'checkpoint-bike.pt'
            path = F"/content/drive/My Drive/Google_Colab_Local/{model_save_name}"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'class_to_idx': train_dataset.class_to_idx,
                }, path)

        else:
            print((f"Minimum Loss This Epoch: {min_loss}... ",
                   f"Running Loss on Test Set: {running_test_loss}... ",
                   f"Minimum Loss has NOT been updated... "
                   f"Model was NOT Saved"))             

    model.train()

Epoch 1/25...  Step 63/752..  Step Loss 0.236..  Accuracy 0.918.. 
('Minimum Loss This Epoch: 20...\n ', 'Running Loss on Test Set: 2.8908329139230773...\n ', "Minimum Loss has been updated... \nModel was Saved to 'checkpoint-bike.pt'")
Epoch 2/25...  Step 126/752..  Step Loss 0.223..  Accuracy 0.925.. 
('Minimum Loss This Epoch: 2.8908329139230773... ', 'Running Loss on Test Set: 3.260798103554407... ', 'Minimum Loss has NOT been updated... Model was NOT Saved')
Epoch 3/25...  Step 189/752..  Step Loss 0.226..  Accuracy 0.922.. 
('Minimum Loss This Epoch: 2.8908329139230773... ', 'Running Loss on Test Set: 3.507865780143766... ', 'Minimum Loss has NOT been updated... Model was NOT Saved')
Epoch 4/25...  Step 252/752..  Step Loss 0.230..  Accuracy 0.918.. 
('Minimum Loss This Epoch: 2.8908329139230773...\n ', 'Running Loss on Test Set: 2.597514481341932...\n ', "Minimum Loss has been updated... \nModel was Saved to 'checkpoint-bike.pt'")
Epoch 5/25...  Step 315/752..  Step Loss 0.227..

KeyboardInterrupt: ignored

In [5]:
test_loss = 0
accuracy = 0

model.eval()
with torch.no_grad():
    for inputs, labels in valid_loaders:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()

        #calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(valid_loaders):.3f}.. ",
      f"Test accuracy: {accuracy/len(valid_loaders):.3f}")
model.train()

Test loss: 2.021..  Test accuracy: 0.406


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

###Training 4 - adjusted batch size - 10 epochs

In [0]:
min_loss = 10

#for i in content/drive/My Drive/Google_Colab_Local/workspace_utils.py.keep_awake(range(5)):

epochs = 10
steps = 0
accuracy = 0
running_loss = 0
#print_every = 50

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loaders:
        #increment steps by 1
        steps += 1

        #transfer inputs and labels to appropriate device (cuda or cpu)
        inputs, labels = inputs.to(device), labels.to(device)

        #zero out optimize gradients
        optimizer.zero_grad()

        #feedforward through network and get result
        logps = model(inputs)

        #calculate loss
        loss = criterion(logps, labels)

        #attribute the loss to individual weights
        loss.backward()

        #backpropagate to update the weights
        optimizer.step()

        #update the running_loss
        running_loss += loss.item()

        #display progess report while running
        total_steps = len(train_dataset)

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        print(f"Epoch {epoch+1}/{epochs}... ",
              f"Step {steps}/{total_steps}.. ",
              f"Step Loss {running_loss/steps:.3f}.. ",
              f"Accuracy {accuracy/steps:.3f}.. ")

    #Begin determining if a save is needed
    test_loss = 0
    iterations = 0
    model.eval()

    with torch.no_grad():
        for inputs, labels, in test_loaders:
            iterations += 1
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)

            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

        running_test_loss = test_loss/iterations

        if running_test_loss < min_loss:
            print((f"Minimum Loss This Epoch: {min_loss}...\n ",
                   f"Running Loss on Test Set: {running_test_loss}...\n ",
                   f"Minimum Loss has been updated... \n"
                   f"Model was Saved to 'checkpoint-bike.pt'"))

            min_loss = running_test_loss

            model_save_name = 'checkpoint-bike.pt'
            path = F"/content/drive/My Drive/Google_Colab_Local/{model_save_name}"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'class_to_idx': train_dataset.class_to_idx,
                }, path)

        else:
            print((f"Minimum Loss This Epoch: {min_loss}... ",
                   f"Running Loss on Test Set: {running_test_loss}... ",
                   f"Minimum Loss has NOT been updated... "
                   f"Model was NOT Saved"))             

    model.train()

Epoch 1/10...  Step 1/752..  Step Loss 1.769..  Accuracy 0.250.. 
Epoch 1/10...  Step 2/752..  Step Loss 1.123..  Accuracy 0.521.. 
Epoch 1/10...  Step 3/752..  Step Loss 0.867..  Accuracy 0.597.. 
Epoch 1/10...  Step 4/752..  Step Loss 0.677..  Accuracy 0.698.. 
Epoch 1/10...  Step 5/752..  Step Loss 0.552..  Accuracy 0.758.. 
Epoch 1/10...  Step 6/752..  Step Loss 0.470..  Accuracy 0.799.. 
Epoch 1/10...  Step 7/752..  Step Loss 0.429..  Accuracy 0.815.. 
Epoch 1/10...  Step 8/752..  Step Loss 0.378..  Accuracy 0.839.. 
Epoch 1/10...  Step 9/752..  Step Loss 0.337..  Accuracy 0.856.. 
Epoch 1/10...  Step 10/752..  Step Loss 0.310..  Accuracy 0.871.. 
Epoch 1/10...  Step 11/752..  Step Loss 0.286..  Accuracy 0.883.. 
Epoch 1/10...  Step 12/752..  Step Loss 0.265..  Accuracy 0.892.. 
Epoch 1/10...  Step 13/752..  Step Loss 0.256..  Accuracy 0.897.. 
Epoch 1/10...  Step 14/752..  Step Loss 0.240..  Accuracy 0.905.. 
Epoch 1/10...  Step 15/752..  Step Loss 0.228..  Accuracy 0.911.. 
Epoc

In [22]:
test_loss = 0
accuracy = 0

model.eval()
with torch.no_grad():
    for inputs, labels in valid_loaders:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()

        #calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(valid_loaders):.3f}.. ",
      f"Test accuracy: {accuracy/len(valid_loaders):.3f}")
model.train()

Test loss: 1.376..  Test accuracy: 0.510


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

###Training 5 - 10 epochs

In [0]:
min_loss = 10

#for i in content/drive/My Drive/Google_Colab_Local/workspace_utils.py.keep_awake(range(5)):

epochs = 10
steps = 0
accuracy = 0
running_loss = 0
#print_every = 50

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loaders:
        #increment steps by 1
        steps += 1

        #transfer inputs and labels to appropriate device (cuda or cpu)
        inputs, labels = inputs.to(device), labels.to(device)

        #zero out optimize gradients
        optimizer.zero_grad()

        #feedforward through network and get result
        logps = model(inputs)

        #calculate loss
        loss = criterion(logps, labels)

        #attribute the loss to individual weights
        loss.backward()

        #backpropagate to update the weights
        optimizer.step()

        #update the running_loss
        running_loss += loss.item()

        #display progess report while running
        total_steps = len(train_dataset)

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        print(f"Epoch {epoch+1}/{epochs}... ",
              f"Step {steps}/{total_steps}.. ",
              f"Step Loss {running_loss/steps:.3f}.. ",
              f"Accuracy {accuracy/steps:.3f}.. ")

    #Begin determining if a save is needed
    test_loss = 0
    iterations = 0
    model.eval()

    with torch.no_grad():
        for inputs, labels, in test_loaders:
            iterations += 1
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)

            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

        running_test_loss = test_loss/iterations

        if running_test_loss < min_loss:
            print((f"Minimum Loss This Epoch: {min_loss}...\n ",
                   f"Running Loss on Test Set: {running_test_loss}...\n ",
                   f"Minimum Loss has been updated... \n"
                   f"Model was Saved to 'checkpoint-bike.pt'"))

            min_loss = running_test_loss

            model_save_name = 'checkpoint-bike.pt'
            path = F"/content/drive/My Drive/Google_Colab_Local/{model_save_name}"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'class_to_idx': train_dataset.class_to_idx,
                }, path)

        else:
            print((f"Minimum Loss This Epoch: {min_loss}... ",
                   f"Running Loss on Test Set: {running_test_loss}... ",
                   f"Minimum Loss has NOT been updated... "
                   f"Model was NOT Saved"))             

    model.train()

Epoch 1/10...  Step 1/752..  Step Loss 1.107..  Accuracy 0.375.. 
Epoch 1/10...  Step 2/752..  Step Loss 0.917..  Accuracy 0.521.. 
Epoch 1/10...  Step 3/752..  Step Loss 0.688..  Accuracy 0.639.. 
Epoch 1/10...  Step 4/752..  Step Loss 0.568..  Accuracy 0.708.. 
Epoch 1/10...  Step 5/752..  Step Loss 0.491..  Accuracy 0.750.. 
Epoch 1/10...  Step 6/752..  Step Loss 0.444..  Accuracy 0.778.. 
Epoch 1/10...  Step 7/752..  Step Loss 0.389..  Accuracy 0.810.. 
Epoch 1/10...  Step 8/752..  Step Loss 0.352..  Accuracy 0.833.. 
Epoch 1/10...  Step 9/752..  Step Loss 0.322..  Accuracy 0.852.. 
Epoch 1/10...  Step 10/752..  Step Loss 0.302..  Accuracy 0.867.. 
Epoch 1/10...  Step 11/752..  Step Loss 0.275..  Accuracy 0.879.. 
Epoch 1/10...  Step 12/752..  Step Loss 0.256..  Accuracy 0.889.. 
Epoch 1/10...  Step 13/752..  Step Loss 0.240..  Accuracy 0.897.. 
Epoch 1/10...  Step 14/752..  Step Loss 0.228..  Accuracy 0.905.. 
Epoch 1/10...  Step 15/752..  Step Loss 0.214..  Accuracy 0.911.. 
Epoc